# Yada Deconvolution

- Run the following cells for deconvolution using Yada.
- Example files are provided in data folder.

In [ ]:
#Import prerequisites
!pip install tslearn
!rm -r Yada
!git clone https://github.com/zurkin1/Yada.git
!mv Yada/* .

     |████████████████████████████████| 778kB 4.4MB/s 
rm: cannot remove 'Yada': No such file or directory
Cloning into 'Yada'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 230 (delta 35), reused 48 (delta 17), pack-reused 159
Receiving objects: 100% (230/230), 53.69 MiB | 23.78 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [ ]:
!cp data/GBM/pure.csv ./data/pure.csv

## Configure input files (sample files are provided).

In [ ]:
mix = 'data/GBM/mix.csv' #This is the mixture file in the format: columns: mix1, mix2, ..., rows: HUGO gene names.

## Select deconvolution methods (or leave default values if using PBMC cell types).

In order to simplify usage we tried not to require different platform, normalizations and scale parameters. Any such configuration can be added to the method.

In [ ]:
from yada import *

#cells_pbmc14 = ['memory.B.cells', 'naive.B.cells', 'memory.CD4.T.cells', 'naive.CD4.T.cells', 'regulatory.T.cells', 'memory.CD8.T.cells', 'naive.CD8.T.cells', 'NK.cells', 'neutrophils', 'monocytes', 'myeloid.dendritic.cells', 'macrophages', 'fibroblasts', 'endothelial.cells']
cells_brain = ['astrocytes','endothelial','oligodendrocyte','microglia', 'neurons','oligo-precursor']
methods = [
            [10, #Number of runs.
             dtw_deconv, #Method.
             cells_brain,
             'pure', #Reference matrix name. Our sample mix data is RNA-seq TPM normalized so we used an appropriate matrix.
            ]
        ]

## Run deconvolution.

In [ ]:
from IPython.display import FileLink

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
run_deconv(mix, methods)

Deconvolution, num_mixes: 16
dtw_deconv, pure
 90%

,astrocytes,endothelial,oligodendrocyte,microglia,neurons,oligo-precursor
0,0.000055,0.343743,0.000000,0.016401,0.015267,0.360145
1,0.000281,0.342476,0.003175,0.031443,0.038858,0.343515
2,0.015177,0.000146,0.062364,0.000052,0.719958,0.090205
3,0.002655,0.444776,0.015403,0.041123,0.110684,0.366308
4,0.016348,0.002395,0.070159,0.000301,0.774769,0.102889
5,0.005581,0.382097,0.032466,0.082038,0.238522,0.357484
6,0.018558,0.001337,0.072083,0.000333,0.884585,0.111977
7,0.004405,0.411838,0.018418,0.036300,0.198378,0.377027
8,0.000867,0.486296,0.007012,0.032368,0.056493,0.566950
9,0.015410,0.003997,0.065722,0.000145,0.725790,0.093118


In [ ]:
from IPython.display import FileLink, FileLinks

FileLink('data/results.csv')

/content/data/results.csv